# Task 04 - Maximum Likelihood Parameter Estimation
## Pattern Recognition and Machine Learning

Copy and import needed files/methods from previous assignment to this directory. 
Adding path to the previous assignment is not sufficient. Upload system
requires your code to be self contained.

In [ ]:
# uncomment following for interactive matplotlib
# %matplotlib notebook

from mle import *
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## Part 1

In [ ]:
step_cardinality = 10
max_cardinality = 500
cardinalities = np.expand_dims(np.arange(step_cardinality,max_cardinality,step_cardinality), 0)
n = cardinalities.size

var_mu_rec = np.zeros_like(cardinalities, dtype=np.float64)
var_sigma_rec = np.zeros_like(cardinalities, dtype=np.float64)

for idx in range(n):    
    # Compute the variance of the estimations for a fix cardinality
    var_mu_rec[0,idx], var_sigma_rec[0,idx] = mle_variance(cardinalities[0,idx])

#### Visualisation

In [ ]:
plt.figure()
plt.plot(cardinalities.T, var_mu_rec.T, linewidth=3)
plt.plot(cardinalities.T, var_sigma_rec.T, linewidth=3)
plt.ylim([0, 0.06])
plt.legend(['Mean','Standard deviation'])
plt.xlabel('Cardinality of traning set')
plt.ylabel('Variance')
plt.title('MLE variances')
plt.grid('on')
plt.savefig('mle_variances.png')

## Part 2

#### Load data from *.mat files

In [ ]:
def unwrap(data):
    """
    Simple "hack" for preparing data from *.mat files
    """
    try:
        while (len(data) == 1) and (len(data.shape) > 0):
            data = data[0]
        for key in list(data.dtype.names):
            data[key] = unwrap(data[key])
    except:
        pass
    return data

def ndarray2dict(data, indexes=None):
    outputs = {}
    for key in list(data.dtype.names):
        value = unwrap(data[key])
        try:
            if len(value.shape) > 0:
                value = np.atleast_2d(value)
        except:
            pass
        outputs[key] = value
    if indexes is not None:
        for key in indexes:
            outputs[key] -= 1
    return outputs
    
data = scipy.io.loadmat("data_33rpz_04_mle.mat")
Alphabet = np.array(list(unwrap(data["Alphabet"])))
tst = ndarray2dict(unwrap(data["tst"]), ['labels'])
trn_20 = ndarray2dict(unwrap(data["trn_20"]), ['labels'])
trn_200 = ndarray2dict(unwrap(data["trn_200"]), ['labels'])
trn_2000 = ndarray2dict(unwrap(data["trn_2000"]), ['labels'])
trn_sets = {'20': trn_20, '200': trn_200, '2000': trn_2000}

In [ ]:
# Select the training set
picked_set = '2000'
trn_set = trn_sets[picked_set]

In [ ]:
# Computing features vectors (trainning)
x = compute_measurement_lr_cont(trn_set['images'])

In [ ]:
# Estimate prior probabilities
prior_A = estimate_prior(0,trn_set['labels'])
prior_C = estimate_prior(1,trn_set['labels'])

In [ ]:
# Splitting the trainning data into into classes
x_A = x[0:1, trn_set['labels'][0] == 0]
x_C = x[0:1, trn_set['labels'][0] == 1]

In [ ]:
# Computing Gaussian models of Maximal Likelihood
DA = dict()
DC = dict()

DA['Mean'], DA['Sigma'] = mle_normal(x_A)
DC['Mean'], DC['Sigma'] = mle_normal(x_C)

DA['Prior'] = prior_A
DC['Prior'] = prior_C

#### Visualisation

Plotting L VS sigma

In [ ]:
sigmas = np.array([range(200,1000+1)], dtype=np.float64)
L, maximizer_sigma, max_L = loglikelihood_sigma(x_C,DC,sigmas)

# Plotting the likelihood as a function of Sigma
plt.figure()
plt.plot(sigmas.T,L.T)
plt.grid('on')
plt.title('Loglikelihood for varying sigma (class C, $trn_{' + picked_set + '}$)')
plt.xlabel('$\sigma$')
plt.ylabel('L($\sigma$)')
plt.plot(maximizer_sigma, max_L, 'r+', markersize=15)
plt.plot([DC['Sigma'], DC['Sigma']], [np.min(L), max_L], 'g')
plt.savefig('loglikelihood{}.png'.format(picked_set))

#### Ploting the aproximated density functions

In [ ]:
limit = 4000
num_bins = 20
dom = np.arange(-limit,limit,dtype=np.float64)
raise NotImplementedError("You have to implement the rest.")

In [ ]:
# Compute histograms
hist_A, bin_centers_A = np.histogram(x_A,num_bins)
hist_C, bin_centers_C = np.histogram(x_C,num_bins)

# Normalize histograms
hist_A = hist_A / (np.sum(hist_A) * (bin_centers_A[1]-bin_centers_A[0]))
hist_C = hist_C / (np.sum(hist_C) * (bin_centers_C[1]-bin_centers_C[0]))

#### Visualisation for letter A

In [ ]:
# Plot histograms
plt.figure()
plt.bar(bin_centers_A[:-1], hist_A, width=bin_centers_A[1]-bin_centers_A[0] , color='y', edgecolor='k')
plt.legend([None])
plt.grid('on')
plt.title('Densities functions class A')
plt.savefig('mle_estimatesA.png')

# Overlap estimated distributions
raise NotImplementedError("You have to implement the rest.")

#### Visualisation for letter C

In [ ]:
raise NotImplementedError("You have to implement the rest.")

## Estimating the optimal bayesian strategy using the aprox. p.d.s

In [ ]:
raise NotImplementedError("You have to implement the rest.")

# Computing features vectors (test data)
xtst = ...

# classify images
q_x = ...

# classification error
error = ...
print('Error: {:.2f} %'.format(error * 100))

#### Displaying images

In [ ]:
show_classification(tst['images'], x, 'AC')
plt.savefig('mle_classif{}.png'.format(picked_set))